In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import tarfile
import urllib.request

In [ ]:
def load_housing_data():
  tarball_path = Path('datasets/housing.tgz')
  if not tarball_path.is_file():
    Path('datasets').mkdir(parents=True, exist_ok=True)
    url = "https://github.com/ageron/data/raw/main/housing.tgz"
    urllib.request.urlretrieve(url, tarball_path)
    with tarfile.open(tarball_path) as tarball_file:
      tarball_file.extractall(path='datasets')
  return pd.read_csv(Path('datasets/housing/housing.csv'))

In [ ]:
housing_data = load_housing_data()

In [ ]:
housing_data.head()

In [ ]:
housing_data.info()

In [ ]:
housing_data["ocean_proximity"].value_counts()

In [ ]:
housing_data.describe()

In [ ]:
housing_data.hist(bins=50, figsize=(12, 8));

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_set, test_set = train_test_split(housing_data, test_size=0.2, random_state=42)

In [ ]:
len(train_set)

In [ ]:
len(test_set)

In [ ]:
housing_data["income_cat"] = pd.cut(
    housing_data["median_income"],
    bins=[0, 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)

In [ ]:
housing_data["income_cat"].value_counts().sort_index().plot.bar(rot=0, grid=True);

In [ ]:
new_train_set, new_test_set = train_test_split(
    housing_data, 
    test_size=0.2,
    stratify=housing_data["income_cat"],
    random_state=42
)

In [ ]:
new_test_set["income_cat"].value_counts() / len(new_test_set)

# Let's Visualize our Data 

In [ ]:
housing_data.plot(kind="scatter", x="longitude", y="latitude", grid=True);

In [ ]:
housing_data.plot(kind="scatter", x="longitude", y="latitude", grid=True, alpha=0.2);

In [ ]:
housing_data.plot(kind="scatter", x="longitude", y="latitude", grid=True,
    s=housing_data["population"] / 100, label="population",
    c="median_house_value", cmap="jet", colorbar=True,
    legend=True, sharex=False, figsize=(10, 7));

# Correlation Coefficient

In [ ]:
housing_num = housing_data.select_dtypes(include=[np.number])

In [ ]:
corr_matrix = housing_num.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms",
"housing_median_age"]

In [ ]:
scatter_matrix(housing_data[attributes], figsize=(12, 8));

In [ ]:
housing_data.plot(kind="scatter", x="median_income", y="median_house_value",
alpha=0.1, grid=True);

# Let's Experiment

In [ ]:
housing_data["rooms_per_house"] = housing_data["total_rooms"] / housing_data["households"]
housing_data["bedrooms_ratio"] = housing_data["total_bedrooms"] / housing_data["total_rooms"]
housing_data["people_per_house"] = housing_data["population"] / housing_data["households"]

In [ ]:
housing_num = housing_data.select_dtypes(include=[np.number])

In [ ]:
corr_matrix = housing_num.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

# Let's Prepare the Data for Machine Learning Model

In [ ]:
housing = new_train_set.drop("median_house_value", axis=1)
housing_labels = new_train_set["median_house_value"].copy()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy="median")

In [ ]:
housing_num = housing.select_dtypes(include=[np.number])

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
imputed_df = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
imputed_df.info()

# Handling Text & Categorical Attributes

In [ ]:
housing_cat = housing[["ocean_proximity"]]

In [ ]:
housing_cat.head(10)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
cat_encoder = OneHotEncoder()

In [ ]:
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

In [ ]:
housing_cat_1hot = housing_cat_1hot.toarray()

In [ ]:
encoded_df = pd.DataFrame(housing_cat_1hot, columns=cat_encoder.get_feature_names_out(), index=housing_cat.index)

In [ ]:
encoded_df.head()

# Feature Scaling

In [ ]:
housing.hist(bins=50, figsize=(12, 8));

In [ ]:
plt.hist(housing["population"], bins=50);

In [ ]:
plt.hist(np.log(housing["population"]), bins=50);

In [ ]:
scale_attributes = ["total_rooms", "total_bedrooms", "population", "households", "median_income"]

In [ ]:
rem_attributes = ["latitude", "longitude", "housing_median_age"]

In [ ]:
column_names = rem_attributes + scale_attributes

In [ ]:
rem_df = imputed_df[rem_attributes].copy()

In [ ]:
scaled_df = imputed_df[scale_attributes].copy()

In [ ]:
df_log = np.log(scaled_df)

In [ ]:
nor_df = rem_df.join(df_log)

In [ ]:
nor_df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_df = scaler.fit_transform(nor_df)

In [ ]:
scaled_df_ = pd.DataFrame(scaled_df, columns=column_names, index=nor_df.index)

In [ ]:
scaled_df_.head()

In [ ]:
df_to_train = scaled_df_.join(encoded_df)

# Training a Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
df_to_train.head()

In [ ]:
lr.fit(df_to_train, housing_labels)

In [ ]:
df_to_train.head()

In [ ]:
housing_predictions = lr.predict(df_to_train)

# Let's Evaluate the Model

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)

In [ ]:
rmse